# Welcome

Welcome to this interactive notebook showcasing the new features of C# 13. In this notebook, we will explore some of the exciting enhancements and additions to the C# language.
You will find code examples and explanations that demonstrate how these new features can be used to write more efficient and expressive code. Let's dive in and see what's new in C# 13!
This showcase will follow the [official post](https://learn.microsoft.com/en-us/dotnet/csharp/whats-new/csharp-13) about the newest features in C#, if you want furhter information, you will find them there!

## Intro
Before we dive into the new features, lets get familiar with Polyglot notebooks. If your are familiar with Jupiter notebooks, you can probably skip this part. For all new to notebooks, let me explain the basics:
In a notebook, you can tell an interactive journey using text and code. Below you will find your first code cell, update the variable `name`, to hold your name.

In [11]:
var name = "Bjarne";

As you can see, the variable is not scoped to a single cell. Instead we can use it in the cell above to welcome you.
Congrats, you just solved your first cell!. Now check the output below. 

In [12]:
Console.WriteLine("Hello, " + name + "!");

Hello, Bjarne!


# Params Collection
You probably heard of them, but rarely used them: Params collections. Let's refresh our memories and fix the code using params collection.

In [19]:
public void PrintStrings(string[] strings)
{
    foreach (var s in strings)
    {
        Console.WriteLine(s);
    }
}

PrintStrings("Donald", "Mickey", "Goofy");

Donald
Mickey
Goofy


The `params` modifier allows us to create a method that takes an arbitrary number of parameters of the same type wihthout using an array to call the method. In our example, we can just pass three strings.
In C# 12, `params` can only be applied to arrays, not `ImmutableArray<T>` or even `Span<T>`. This changed in C#13 where we can use params for any valid collection epxression type such as `Span<T>`, `ICollection<T>`, `IEnumerable` and much more.
 

In [4]:
public void PrintStrings(params ReadOnlySpan<string> strings)
{
    foreach (var s in strings)
    {
        Console.WriteLine(s);
    }
}

PrintStrings("Donald", "Mickey", "Goofy");  

Error: (1,26): error CS9202: Feature 'params collections' is not available in C# 12.0. Please use language version 13.0 or greater.

# New Lock Object
If you work with multiple Threads / Taks in C#, you may encounter the situation in which a special part of your logic should not be executed in parallel.
For example a with a dictionary, one should not delete and access the same key at the same point in time. Instead we want execute the different actions after each other and not in parallel. This can be achieved with the `lock(x){}`expression:

In [5]:
var lockObject = new Object();

lock(lockObject)
{
    Console.WriteLine("Only one thread at a time.");
}

Only one thread at a time.


Up to C# 12, any `Object` could be used as a `lockObject`. When a given Thread entered the `lock` block, it would use the `lockObject` to mark that is has entered the locked block, preventing any other Thread from entering the block with the same object.
Internally, the compiler would transpile the `lock`statement into:
```C#
object __lockObj = x;
bool __lockWasTaken = false;
try
{
    System.Threading.Monitor.Enter(__lockObj, ref __lockWasTaken);
    Console.WriteLine("Only one thread at a time.");
}
finally
{
    if (__lockWasTaken) System.Threading.Monitor.Exit(__lockObj);
}

```
Using `System.Threading.Monitor.Enter` for handling the different Threads, blocking them until the previous Thread called `Exit.


C# 13 introduces a new special Type `System.Threading.Lock` to be used as the `lockObject`: 

In [6]:
var lockObject = new System.Threading.Lock();

lock(lockObject)
{
    Console.WriteLine("Only one thread at a time.");
}

Error: (3,6): error CS9202: Feature 'Lock object' is not available in C# 12.0. Please use language version 13.0 or greater.

If the lock statement is used with the new Type, the compiler generates a different result, based on a disposbale scope acquired by the lock. If the scope is disposed, another thread can enter the locked scoped. Allowing a more streamline approach at thrad synchronization based on known patterns.

In [ ]:
using (lockObject.EnterScope())
{
    Console.WriteLine("Only one thread at a time.");
}

# New Escape Sequence 
C# 13 introduces a new character literal. While character literals may not ring a bell, you a probably familiar with the character literal for a new line `\n`. In the newest version of C#, you can use `\e` as the escape sequence for the `ESCAPE` character, which was previously achieved via `\u001b`.
The `ESCAPE`character allows the usage of the old [ANSI-Escape-Codes](https://en.wikipedia.org/wiki/ANSI_escape_code), allowing different formattings of the text:

In [3]:
Console.WriteLine("Hello, in regular Text");
Console.WriteLine("\u001b[1mHello, in bold Text\u001b[0m");
Console.WriteLine("\u001b[4mHello, in underlined text\u001b[0m");

Hello, in regular Text
Hello, in bold Text
Hello, in underlined text


With the new character literal `\e`, these become easier to use:

In [4]:
Console.WriteLine("Hello, in regular Text");
Console.WriteLine("\e[1mHello, in bold Text\e[0m");
Console.WriteLine("\e[4mHello, in underlined text\e[0m");

Error: (2,20): error CS9202: Feature 'string escape character' is not available in C# 12.0. Please use language version 13.0 or greater.
(2,44): error CS9202: Feature 'string escape character' is not available in C# 12.0. Please use language version 13.0 or greater.
(3,20): error CS9202: Feature 'string escape character' is not available in C# 12.0. Please use language version 13.0 or greater.
(3,50): error CS9202: Feature 'string escape character' is not available in C# 12.0. Please use language version 13.0 or greater.

# Implicit Index Access
C# 8 introduced the two types `System.Index` and `System.Range` allowing easy indexing and enumration of collections:

In [10]:
public void PrintArray<T>(string title, T[] array)
{
    Console.Write($"{title}: ");
    foreach (var item in array)
    {
        Console.Write($"{item}, ");
    }
    Console.WriteLine();
}

In [12]:
int[] numbers = [1,2,3,4,5,6,7,8,9,10];

var myNumbers = numbers[2..5];
PrintArray(nameof(myNumbers), myNumbers);

var lastTwo = numbers[^2..];
PrintArray(nameof(lastTwo), lastTwo);

myNumbers: 3, 4, 5, 
lastTwo: 9, 10, 


In C# 13, we can now use the *from the end* index operator `^`, for object initialization

In [13]:
var coolNames = new string[4];

coolNames = {
    [^1] = "Mickey",
    [^2] = "Donald",
    [^3] = "Goofy",
    [^4] = "Pluto"
};
PrintArray(nameof(coolNames), coolNames);

Error: (3,13): error CS1525: Invalid expression term '{'
(3,13): error CS1002: ; expected
(4,20): error CS1002: ; expected
(4,20): error CS1513: } expected
(5,20): error CS1002: ; expected
(5,20): error CS1513: } expected
(6,19): error CS1002: ; expected
(6,19): error CS1513: } expected
(7,19): error CS1002: ; expected

# Use `ref` and `unsafe` in iterators and `async` methods
If you are unfamiliar with `ref, unsafe` and `ref struct`, I recommend reading [this explanaition](https://okyrylchuk.dev/blog/understanding-struct-vs-ref-struct-in-csharp/) from Oleg Kyrylchuk.

As a short summary, while a `struct` type can be boxed and thus, moved to the heap, a `ref struct` cant be moved to the heap.

C# 13 allows the usage of `ref` and `unsafe` within iterators and async methods, as long as they stay within the `await` boundary or do not cross a `yield return`.

For example, one is now able to define or use a `Span<T>` which is a `ref struct` inside an `async` method.

In [1]:
public async Task<bool> MyAsyncMethod()
{
    await Task.Delay(1000);
    var span = "Hello, World!".AsSpan();
    var result = span.EndsWith("World");
    return result;
}

Error: (4,5): error CS9202: Feature 'ref and unsafe in async and iterator methods' is not available in C# 12.0. Please use language version 13.0 or greater.

# `allows ref struct`

# `ref struct` interfaces

As stated before, some cosntraints regarding `ref` and `unsafe` where lifted. In C# 13, `ref structs` are now able to implement interfaces. In contrast to types which can be moved to heap, such `ref structs`can not be cast to the interface, as this would box them and thus, move them to the heap.

In [5]:
interface IMyInterface
{
    void MyMethod();
}

ref struct MyRefStruct : IMyInterface
{
    public void MyMethod()
    {
        Console.WriteLine("MyMethod called.");
    }
}

void TestMyRefStruct()
{
    var myRefStruct = new MyRefStruct();
    myRefStruct.MyMethod();

    // Not allowed, as ref structs cannot be boxed
    IMyInterface myInterface = myRefStruct;
    
}


Error: (6,26): error CS9202: Feature 'ref struct interfaces' is not available in C# 12.0. Please use language version 13.0 or greater.
(20,32): error CS0029: Cannot implicitly convert type 'MyRefStruct' to 'IMyInterface'

# More `partial` Members

# Overload resolution priority

# Introducing the `field` Keyword
This new feature is still in preview in C#13 and can be enabled by setting the `<LangVersion>` to `preview` within your project file. Enabling this feature introduces a new token `field`, which may introduce a breaking change in you codebase, as `field` is now a reserved token of C#. As a mitigashiatíon, you can rename your usage of `field` into `@field` or `this.field`.

The new keyword can be used in getters and setters to access the compiler synthesized backing field of a property:

In [2]:
class Ticket {
    public int Price {
        get;
        set => field = (value > 0) 
            ? value 
            : throw new ArgumentOutOfRangeException("Price must be positive");;
    }
}

Error: (6,75): error CS1014: A get or set accessor expected